In [ ]:
#%matplotlib widget

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import scipy.stats as st
from pathlib import Path
%matplotlib inline


In [16]:
#pull in CSV df - 2019 through 2021 - vehicle csv
csv_path2019 = Path("Resources/vehicle_2019.csv")
csv_path2020 = Path("Resources/vehicle_2020.csv")
csv_path2021 = Path("Resources/vehicle_2021.csv")

vehicle2019_df = pd.read_csv(csv_path2019, encoding='cp1252', low_memory=False)
vehicle2020_df = pd.read_csv(csv_path2020, encoding='cp1252', low_memory=False)
vehicle2021_df = pd.read_csv(csv_path2021, encoding='cp1252', low_memory=False)

#Pull in CSV df - 2019 through 2021 - Person csv
csv2_path2019 = Path("Resources/Person_2019.csv")
csv2_path2020 = Path("Resources/Person_2020.csv")
csv2_path2021 = Path("Resources/Person_2021.csv")

person2019_df = pd.read_csv(csv2_path2019, encoding='cp1252', low_memory=False)
person2020_df = pd.read_csv(csv2_path2020, encoding='cp1252', low_memory=False)
person2021_df = pd.read_csv(csv2_path2021, encoding='cp1252', low_memory=False)

# Extract the "INJ_SEV" and "INJ_SEVNAME" columns from person2019_df
injuries2019 = person2019_df[['INJ_SEV', 'INJ_SEVNAME']]
injuries2020 = person2020_df[['INJ_SEV', 'INJ_SEVNAME']]
injuries2021 = person2021_df[['INJ_SEV', 'INJ_SEVNAME']]

# Add the extracted columns to the fixed DataFrames
vehicle2019_df[['INJ_SEV', 'INJ_SEVNAME']] = injuries2019
vehicle2020_df[['INJ_SEV', 'INJ_SEVNAME']] = injuries2020
vehicle2021_df[['INJ_SEV', 'INJ_SEVNAME']] = injuries2021

#Clean CSV df - 2019 through 2021 - vehicle csv
cleaned2019 = vehicle2019_df[['STATE','STATENAME','MONTHNAME','BODY_TYP','DEATHS','ACC_TYPE','OWNER','MAK_MODNAME','HARM_EVNAME', 'MAN_COLLNAME',
                               'VNUM_LAN','INJ_SEV','INJ_SEVNAME']]
cleaned2020 = vehicle2020_df[['STATE','STATENAME','MONTHNAME','BODY_TYP','DEATHS','ACC_TYPE','OWNER','MAK_MODNAME','HARM_EVNAME', 'MAN_COLLNAME',
                               'VNUM_LAN','INJ_SEV','INJ_SEVNAME']]
cleaned2021 = vehicle2021_df[['STATE','STATENAME','MONTHNAME','BODY_TYP','DEATHS','ACC_TYPE','OWNER','MAK_MODNAME','HARM_EVNAME', 'MAN_COLLNAME',
                               'VNUM_LAN','INJ_SEV','INJ_SEVNAME']]


# Renaming columns and filtering them
new_names = {"HARM_EVNAME": "Accident Type", "MAN_COLLNAME": "Vehicle Crash Description", "VNUM_LAN": "Lanes Involved",
             "BODY_TYP": "Vehicle Type ID", "STATENAME": "State Name", "MONTHNAME": "Month Name", "BODY_TYPNAME": "Body Type",
             "DEATHS":"Deaths", "OWNER": "Owner", "MAK_MODNAME": "Make and Model","INJ_SEV": "Injury ID","INJ_SEVNAME":"Injury Type"}
fixed2019 = cleaned2019.rename(columns=new_names)
fixed2020 = cleaned2020.rename(columns=new_names)
fixed2021 = cleaned2021.rename(columns=new_names)

# Filters by potential lane splitting or filtering motorcycles exclusively
filtered2019 = fixed2019[(fixed2019['Vehicle Type ID'] == 80) & (fixed2019['Lanes Involved'] != 1)]
filtered2020 = fixed2020[(fixed2020['Vehicle Type ID'] == 80) & (fixed2020['Lanes Involved'] != 1)]
filtered2021 = fixed2021[(fixed2021['Vehicle Type ID'] == 80) & (fixed2021['Lanes Involved'] != 1)]

#



In [17]:
#clean data for CA (Legal) -  years 2019 through 2021 
legal_2019_df = filtered2019[(filtered2019['STATE']== "6")]
legal_2020_df = filtered2020[(filtered2020['STATE']== "6")]
legal_2021_df = filtered2021[(filtered2021['STATE']== "6")]

#clean data for (Somewhat Legal) - years 2019 through 2021
somewhat_legal_2019_df = filtered2019[filtered2019['STATE'].isin(["15","49"])]  
somewhat_legal_2020_df = filtered2020[filtered2020['STATE'].isin(["15","49"])]   
somewhat_legal_2021_df = filtered2021[filtered2021['STATE'].isin(["15","49"])] 

#clean data for (Illegal) - years 2019 through 2021 
illegal_2019_df = filtered2019[filtered2019['STATE'].isin(["30","35","48","40","29","5","28","20","39","54","37","34","43","52"])] 
illegal_2020_df = filtered2020[filtered2020['STATE'].isin(["30","35","48","40","29","5","28","20","39","54","37","34","43","52"])] 
illegal_2021_df = filtered2021[filtered2021['STATE'].isin(["30","35","48","40","29","5","28","20","39","54","37","34","43","52"])] 

#clean data for (Not Mentioned) - years 2019 through 2021 
not_mentioned_2019_df = filtered2019[filtered2019['STATE'].isin(["30","35","48","40","29","5","28","20","39","54","37","34","43","52"])] 
not_mentioned_2020_df = filtered2020[filtered2020['STATE'].isin(["30","35","48","40","29","5","28","20","39","54","37","34","43","52"])] 
not_mentioned_2021_df = filtered2021[filtered2021['STATE'].isin(["30","35","48","40","29","5","28","20","39","54","37","34","43","52"])] 

#


In [31]:
#Filter the dataframes for each year for appropriate columns

df_list = [vehicle2019_df, vehicle2020_df, vehicle2021_df]

filtered_df_list=[]

for df in df_list:

    cleaned_df = df[['STATE','BODY_TYP','MONTH','INJ_SEV','INJ_SEVNAME','OWNER']]
    motorcycle_df = cleaned_df.loc[cleaned_df['BODY_TYP'].between(80, 87)]
    summer_df = motorcycle_df.loc[(motorcycle_df['MONTH'] >= 4) & (motorcycle_df['MONTH'] <= 8)]
    final_filter_df = summer_df.loc[(summer_df['OWNER'] != 0) & (summer_df['INJ_SEV'] > 0)]

    filtered_df_list.append(final_filter_df)
    # Iterate through the filtered DataFrames and display them
for idx, df in enumerate(filtered_df_list):
    print(f"Filtered DataFrame {idx + 1}:")
    print(df)




Filtered DataFrame 1:
       STATE  BODY_TYP  MONTH  INJ_SEV                   INJ_SEVNAME  OWNER
281        1        82      5        4              Fatal Injury (K)      1
295        1        80      4        3  Suspected Serious Injury (A)      1
299        1        80      4        4              Fatal Injury (K)      1
304        1        80      4        1           Possible Injury (C)      2
319        1        80      4        3  Suspected Serious Injury (A)      2
...      ...       ...    ...      ...                           ...    ...
51535     56        80      6        4              Fatal Injury (K)      1
51538     56        80      7        1           Possible Injury (C)      1
51541     56        80      7        4              Fatal Injury (K)      1
51559     56        80      8        4              Fatal Injury (K)      1
51561     56        80      7        4              Fatal Injury (K)      1

[2110 rows x 6 columns]
Filtered DataFrame 2:
       STATE  BODY_